In [1]:
from db_credentials	import dwh_db_connection_params
from database import Database
from variables import sourcepath
import os
import pandas as pd
import numpy as np

In [2]:
import roman

In [3]:
pap=pd.read_csv(os.path.join(sourcepath, 'papers_final.csv'))[['journal', 'volume', 'issue', 'publisher', 'place']]
ref=pd.read_csv(os.path.join(sourcepath, 'unique_references.csv'))[['journal', 'volume', 'issue', 'publisher', 'place']]

In [9]:
print(ref.shape)
print(pap.shape)

(192003, 5)
(7479, 5)


In [4]:
all=ref.append(pap, ignore_index=True)
all.shape

(199482, 5)

In [5]:
all.dropna(axis=0, how='all', inplace=True)
all.shape

(174318, 5)

In [6]:
all.drop_duplicates()
all.shape


(174318, 5)

In [7]:
all.fillna({'journal': 'MISSING', 'volume':0, 'issue': 0, 'publisher': 'MISSING', 'place': 'MISSING'}, inplace=True)

In [8]:
#test
all[pd.to_numeric(all.volume, errors='coerce').isnull()].volume.values

array(['II', 'I', 'IV', 'I', 'I', 'II', 'II', 'Number', '^', 'II', 'A',
       'II', 'I', 'I', 'XXVII', 'XIX', 'IV', 'II', 'II', 'II.', 'II',
       'VI', 'I', 'II', 'II', 'II', 'II', 'II', 'HI', 'II', 'II', 'IV',
       'II', 'II', 'IV', 'IV', 'IV', 'IV', 'IV', 'IV', 'III', 'I', 'II',
       'II', 'II', 'VI', 'chicago', 'I', 'I', 'INFOR', 'II', 'IV', 'II',
       'I', 'II', 'I', 'II', 'II, I', 'XV', 'II', 'SI', 'XI', 'II', 'IV',
       'VII', 'I', 'i', 'IV', 'II', 'XIII', 'XIII', 'XII', 'i', 'I', 'I',
       'II', 'I', 'I', 'III', 'III', 'IV', 'H', 'I', 'VI', 'III', 'IV',
       'II', 'II', 'I', 'd', 'I', 'II', 'I', 'II', 'II', 'II', 'II', 'I',
       'III', 'IV', 'IV', 'I', 'II', 'I', 'I', 'I', 'II', 'III', 'I', 'I',
       'III', 'II', 'VIII. Symantec', 'II', 'I', 'I', 'InformationWeek',
       'IV', 'II', 'I', 'VIII', 'I', 'III', 'I', 'II', 'II', 'V', 'II',
       'II', 'II', 'II', 'IV', 'I', 'S', 'I', 'III', 'III', 'II', 'XLVI',
       'I', 'I', 'II', 'XIII', 'XIV', 'XIV', 'XIV', 

In [29]:
def volume_to_int(volume):
    try:
        vol=int(volume)
    except:
        try:
            vol=roman.fromRoman(volume)
        except:
            vol=0
    if vol>10000:
        vol=0
    return vol

In [30]:
all.volume=all.volume.apply(lambda v: volume_to_int(v))

In [11]:
def issue_to_int(issue):
    try: 
        iss=int(issue)
    except:
        try:
            iss=int(issue[0])
        except:
            iss=0
    return iss

In [31]:
all.issue=all.issue.apply(lambda i: issue_to_int(i))

In [32]:
all.shape

(92784, 5)

In [33]:
all.drop_duplicates(inplace=True)

In [34]:
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92758 entries, 0 to 199459
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   journal    92758 non-null  object
 1   volume     92758 non-null  int64 
 2   issue      92758 non-null  int64 
 3   publisher  92758 non-null  object
 4   place      92758 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.2+ MB


In [39]:
all.issue.max()

883739

In [40]:
all[all.issue>10000]

,journal,volume,issue,publisher,place
35303,New York University Law Review,81,883739,MISSING,MISSING
54598,ComputerWorld,47,57374,MISSING,MISSING
69405,Computerworld,47,50529,MISSING,MISSING
69406,Computerworld,47,54993,MISSING,MISSING
100960,NBER Working Paper Series,0,14128,MISSING,MISSING
114740,National Bureau of Economic Research Working P...,0,10397,MISSING,MISSING


In [38]:
all[all.volume==0]

,journal,volume,issue,publisher,place
24,MISSING,0,0,Pearson Education Inc,Los Angeles
30,MISSING,0,0,Academic Press,London
38,MISSING,0,0,Sage Publications,MISSING
43,Salesforce.com,0,0,MISSING,MISSING
49,BMC Communities,0,0,MISSING,MISSING
...,...,...,...,...,...
191998,Interdisciplinary journal of information,0,0,MISSING,MISSING
191999,Interactive Learning Environments,0,1,MISSING,MISSING
194139,The Journal Of Strategic Information Systems,0,0,MISSING,MISSING
196176,Journal Of The Association For Information Sys...,0,0,MISSING,MISSING


In [19]:
all.issue.max()

883739

In [73]:
all_db=all.iloc[:10]
all_source=all.iloc[:15]

In [74]:
merged=pd.merge(all_source, all_db, how='outer')
print(merged.shape)
merged.head(2)

(15, 5)


,journal,volume,issue,publisher,place
0,Acad. Manag. J,56,5,MISSING,MISSING
1,Inform. Syst. Res,27,4,MISSING,MISSING


In [76]:
pd.concat([merged, all_db]).drop_duplicates(keep=False)

,journal,volume,issue,publisher,place
10,Res.-Technol. Manag,54,5,MISSING,MISSING
11,Organ. Sci,23,4,MISSING,MISSING
12,Res. Policy,34,4,MISSING,MISSING
13,Res. Policy,43,5,MISSING,MISSING
14,MIS Quarterly Executive,9,4,MISSING,MISSING
